# 🧱 1) Scraping ciblé du site ESILV

Scraper robuste qui extrait automatiquement :

- les rubriques principales (école, admissions, formations, etc.)

- les sous-rubriques

- les pages enfants (child pages)

- les informations structurées :

✔ titres

✔ contenus textuels

✔ listes

✔ contacts

✔ URL

✔ full text pour la recherche sémantique

Le scraper produit un JSON propre et structuré, utilisable pour un modèle de langage.

👉 Objectif atteint : `transformation d’un site web en base de connaissances exploitable.`

In [ ]:
import requests
import trafilatura
import xml.etree.ElementTree as ET
import json
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import re
import os
import traceback

In [ ]:
pip install trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.7 MB/s eta 0:00:00


## ANCIENNE VERSION

In [ ]:
import requests
from bs4 import BeautifulSoup
import trafilatura
import json
import os
from urllib.parse import urljoin

START_URLS = {
    "lecole": "https://www.esilv.fr/lecole/",
    "admissions": "https://www.esilv.fr/admissions/",
    "formations": "https://www.esilv.fr/formations/",
    "entreprises-debouches": "https://www.esilv.fr/entreprises-debouches/",
    "recherche": "https://www.esilv.fr/recherche/",
    "international": "https://www.esilv.fr/international/",
}


def get_soup(url):
    r = requests.get(url, timeout=10)
    return BeautifulSoup(r.text, "html.parser")


def extract_main_rubric_data(soup, base_url):
    """Extrait h4, h2, h3 + div content dans la rubrique principale"""

    data = {
        "context_h4": [h.get_text(strip=True) for h in soup.select(".cat_wrapper h4")],
        "titles_h2": [h.get_text(strip=True) for h in soup.select(".cat_wrapper h2")],
        "subtitles_h3": [h.get_text(strip=True) for h in soup.select(".cat_wrapper h3")],
        "paragraphs": [p.get_text(strip=True) for p in soup.select(".cat_wrapper p")],
    }

    return data


def extract_subrubrics(soup, base_url):
    """Extrait les sous-rubriques avec post_header half + post_excerpt half"""

    subrubrics = []

    cards = soup.select(".half")  # Les blocs ESILV sont souvent en half columns

    for card in cards:
        # Nom de la sous-rubrique (dans post_header)
        title_tag = card.select_one(".post-header h2, .post-header h3, .post-header h1")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Résumé court
        excerpt_tag = card.select_one(".post-excerpt p")
        excerpt = excerpt_tag.get_text(strip=True) if excerpt_tag else None

        # Lien vers la sous-page
        link_tag = card.select_one(".post-excerpt a")
        if link_tag and link_tag.get("href"):
            url = urljoin(base_url, link_tag["href"])
        else:
            url = None

        if title or excerpt or url:
            subrubrics.append({
                "title": title,
                "excerpt": excerpt,
                "url": url
            })

    return subrubrics


def extract_contacts(soup):
    """Extraction de la section contacts"""

    contacts_block = soup.select_one(".page_contacts")
    if not contacts_block:
        return None

    contacts = []
    for person in contacts_block.select(".contact"):
        name = person.select_one("h3")
        role = person.select_one("p")
        phone = person.select_one("a[href^='tel']")
        email = person.select_one("a[href^='mailto']")

        contacts.append({
            "name": name.get_text(strip=True) if name else None,
            "role": role.get_text(strip=True) if role else None,
            "phone": phone.get_text(strip=True) if phone else None,
            "email": email.get_text(strip=True) if email else None,
        })

    return contacts


def scrape_page_content(url):
    """Scrape le contenu en texte propre via trafilatura."""
    html = trafilatura.fetch_url(url)
    if html:
        return trafilatura.extract(html)
    return None


def scrape_rubric(name, url):

    print(f"\n=== SCRAPING RUBRIQUE : {name.upper()} ===")

    folder = f"scraping/{name}"
    os.makedirs(folder, exist_ok=True)

    soup = get_soup(url)

    # 1️⃣ Extraire les informations principales de la rubrique
    main_info = extract_main_rubric_data(soup, url)

    # 2️⃣ Extraire les sous-rubriques
    subrubrics = extract_subrubrics(soup, url)

    # 3️⃣ Extraire section contacts
    contacts = extract_contacts(soup)

    # 4️⃣ Scraper le contenu principal en texte propre
    main_text = scrape_page_content(url)

    rubric_json = {
        "rubric": name,
        "url": url,
        "summary": main_info,
        "content_text": main_text,
        "subrubrics": subrubrics,
        "contacts": contacts
    }

    # SAVE main file
    with open(f"{folder}/{name}.json", "w", encoding="utf-8") as f:
        json.dump(rubric_json, f, indent=2, ensure_ascii=False)

    print(f"✔ Rubrique {name} : fichier JSON créé.")

    # 5️⃣ Scraper chaque sous-rubrique dans un fichier séparé
    for sub in subrubrics:
        if not sub["url"]:
            continue

        print("➡️ Sous-rubrique :", sub["title"])

        sub_text = scrape_page_content(sub["url"])

        sub_json = {
            "title": sub["title"],
            "url": sub["url"],
            "excerpt": sub["excerpt"],
            "content_text": sub_text
        }

        filename = sub["title"].lower().replace(" ", "-").replace("é", "e")
        with open(f"{folder}/{filename}.json", "w", encoding="utf-8") as f:
            json.dump(sub_json, f, indent=2, ensure_ascii=False)

    print(f"🎉 Rubrique {name} terminée.\n")


# === EXECUTION ===
for name, url in START_URLS.items():
    scrape_rubric(name, url)

print("\n🏁 SCRAPING COMPLET PAR RUBRIQUE TERMINÉ !")



=== SCRAPING RUBRIQUE : LECOLE ===
✔ Rubrique lecole : fichier JSON créé.
🎉 Rubrique lecole terminée.


=== SCRAPING RUBRIQUE : ADMISSIONS ===
✔ Rubrique admissions : fichier JSON créé.
🎉 Rubrique admissions terminée.


=== SCRAPING RUBRIQUE : FORMATIONS ===
✔ Rubrique formations : fichier JSON créé.
🎉 Rubrique formations terminée.


=== SCRAPING RUBRIQUE : ENTREPRISES-DEBOUCHES ===
✔ Rubrique entreprises-debouches : fichier JSON créé.
🎉 Rubrique entreprises-debouches terminée.


=== SCRAPING RUBRIQUE : RECHERCHE ===
✔ Rubrique recherche : fichier JSON créé.
🎉 Rubrique recherche terminée.


=== SCRAPING RUBRIQUE : INTERNATIONAL ===
✔ Rubrique international : fichier JSON créé.
🎉 Rubrique international terminée.


🏁 SCRAPING COMPLET PAR RUBRIQUE TERMINÉ !


In [ ]:
URL = "https://www.esilv.fr/admissions/"

def scrape_admissions(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    data = {}

    # ─────────────────────────────────────────────
    # INTRO
    # ─────────────────────────────────────────────
    intro_h4 = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    data["intro"] = intro_h4.get_text(strip=True) if intro_h4 else None

    # ─────────────────────────────────────────────
    # LISTE DES SUBRUBRICS (cartes)
    # ─────────────────────────────────────────────
    subrubrics = []

    for card in soup.select(".post_wrapper.one_half"):

        title_tag = card.select_one(".post_header.half h3 a")
        excerpt_tag = card.select_one(".post_excerpt.half p")
        btn = card.select_one(".post_excerpt.half a[href]")

        subrubrics.append({
            "title": title_tag.get_text(strip=True) if title_tag else None,
            "content": excerpt_tag.get_text(strip=True) if excerpt_tag else None,
            "url": urljoin(url, btn["href"]) if btn else None
        })

    # ─────────────────────────────────────────────
    # CONTACTS
    # ─────────────────────────────────────────────
    contacts = []

    for c in soup.select("#page_contacts .one_fifth"):
        name = c.find("h3")
        role = c.find("h4")
        phone_raw = c.find("p")
        email_tag = c.find("a", href=lambda h: h and "mailto" in h)

        phone = None
        if phone_raw:
            phone = phone_raw.get_text(strip=True).split("\n")[0]

        contacts.append({
            "name": name.get_text(separator=" ", strip=True) if name else None,
            "role": role.get_text(separator=" ", strip=True) if role else None,
            "phone": phone,
            "email": email_tag["href"].replace("mailto:", "") if email_tag else None
        })

    # ─────────────────────────────────────────────
    # SECTIONS H2 (avec contenu réassigné)
    # ─────────────────────────────────────────────
    sections = []

    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        title = h2.get_text(strip=True)

        section = {"h2": title, "h3": None, "content": []}

        # H3 juste après ?
        nxt = h2.find_next_sibling()
        if nxt and nxt.name == "h3":
            section["h3"] = nxt.get_text(strip=True)

        # Cas 1 : si le h2 = les procédures → content = subrubrics
        if "procédures" in title.lower():
            section["content"] = subrubrics

        # Cas 2 : si le h2 = Vos contacts → content = contacts
        elif "contacts" in title.lower():
            section["content"] = contacts

        # Cas 3 : sinon → extraire les DIV/MOTS AVANT le prochain h2/hr
        else:
            cur = h2.find_next_sibling()
            while cur and cur.name not in ["h2", "hr"]:
                if cur.name == "div" and cur.get("style"):
                    section["content"].append(cur.get_text(strip=True))
                cur = cur.find_next_sibling()

        sections.append(section)

    data["sections"] = sections

    # ─────────────────────────────────────────────
    # FULL TEXT
    # ─────────────────────────────────────────────
    html = trafilatura.fetch_url(url)
    data["full_text"] = trafilatura.extract(html) if html else None

    return data


# EXECUTION
result = scrape_admissions(URL)
# CLEANING
#result = clean_json_data(result)

# CHUNKING
#result["chunks"] = chunk_rag(result)

with open("admissions_final.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✔ admissions_final.json créé avec succès !")


TypeError: unhashable type: 'dict'

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import trafilatura

URL = "https://www.esilv.fr/admissions/"


# ----------------------------------------------------------
# UTILITAIRE : Nettoyage simple
# ----------------------------------------------------------
def clean(text):
    if not text:
        return None
    return " ".join(text.split()).strip()


# ----------------------------------------------------------
# SCRAPER ADMISSIONS
# ----------------------------------------------------------
def scrape_admissions(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    data = {}

    # Rubrique en titre principal du JSON
    data["rubric"] = "Admissions"

    # ------------------------------------------------------
    # INTRODUCTION <h4>
    # ------------------------------------------------------
    intro_h4 = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    data["intro"] = clean(intro_h4.get_text()) if intro_h4 else None

    # ------------------------------------------------------
    # SUBRUBRICS (cartes)
    # ------------------------------------------------------
    subrubrics = []

    for card in soup.select(".post_wrapper.one_half"):

        title_tag = card.select_one(".post_header.half h3 a")
        excerpt_tag = card.select_one(".post_excerpt.half p")
        btn = card.select_one(".post_excerpt.half a[href]")

        subrubrics.append({
            "title": clean(title_tag.get_text()) if title_tag else None,
            "content": clean(excerpt_tag.get_text()) if excerpt_tag else None,
            "url": urljoin(url, btn["href"]) if btn else None
        })

    # ------------------------------------------------------
    # CONTACTS
    # ------------------------------------------------------
    contacts = []

    for c in soup.select("#page_contacts .one_fifth"):
        name = c.find("h3")
        role = c.find("h4")
        phone_raw = c.find("p")
        email_tag = c.find("a", href=lambda h: h and "mailto" in h)

        phone = None
        if phone_raw:
            phone = clean(phone_raw.get_text()).split(" ")[0]

        contacts.append({
            "name": clean(name.get_text(separator=" ")) if name else None,
            "role": clean(role.get_text(separator=" ")) if role else None,
            "phone": phone,
            "email": email_tag["href"].replace("mailto:", "") if email_tag else None
        })

    # ------------------------------------------------------
    # SECTIONS H2 (avec contenu organisé et nettoyé)
    # ------------------------------------------------------
    sections = []

    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        title = clean(h2.get_text())
        section = {"h2": title}

        # Cherche un éventuel H3 juste après
        nxt = h2.find_next_sibling()
        h3 = clean(nxt.get_text()) if nxt and nxt.name == "h3" else None
        if h3:
            section["h3"] = h3  # on ajoute seulement si non nul

        # Cas : "Les procédures en détails" → mettre subrubrics
        if "procédure" in title.lower():
            section["content"] = subrubrics

        # Cas : "Vos contacts admissions"
        elif "contacts" in title.lower():
            section["content"] = contacts

        # Autres sections → on extrait les div[style]
        else:
            content_list = []
            cur = h2.find_next_sibling()

            while cur and cur.name not in ["h2", "hr"]:
                if cur.name == "div" and cur.get("style"):
                    cleaned = clean(cur.get_text())
                    if cleaned:
                        content_list.append(cleaned)
                cur = cur.find_next_sibling()

            section["content"] = content_list

        # Nettoyer les champs vides
        section = {k: v for k, v in section.items() if v}

        sections.append(section)

    data["sections"] = sections

    # ------------------------------------------------------
    # FULL TEXT POUR LE RAG
    # ------------------------------------------------------
    html = trafilatura.fetch_url(url)
    data["full_text"] = clean(trafilatura.extract(html)) if html else None

    return data


# ----------------------------------------------------------
# EXECUTION
# ----------------------------------------------------------
result = scrape_admissions(URL)

with open("admissions_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✔ admissions_cleaned.json généré avec succès !")

✔ admissions_cleaned.json généré avec succès !


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import trafilatura
import json
import re


def clean_text(t):
    if not t:
        return None
    t = re.sub(r"\s+", " ", t)
    return t.strip()


# ─────────────────────────────────────────────
# SCRAPER STANDARD POUR TOUTE PAGE ENFANT
# ─────────────────────────────────────────────
def scrape_child_page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    result = {
        "url": url,
        "title": None,
        "intro": None,
        "blocks": []
    }

    # h2 principal
    h2 = soup.select_one(".standard_wrapper.cat_wrapper h2")
    if h2:
        result["title"] = clean_text(h2.get_text())

    # h4 introductif
    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        result["intro"] = clean_text(intro.get_text())

    # Sélectionner toutes les zones .one_half (structure commune ESILV)
    for zone in soup.select(".one_half, .one_half.last"):

        block = {
            "h3": None,
            "h4": None,
            "text": [],
            "list": []
        }

        h3 = zone.find("h3")
        h4 = zone.find("h4")

        if h3:
            block["h3"] = clean_text(h3.get_text())

        if h4:
            block["h4"] = clean_text(h4.get_text())

        # Paragraphes
        for p in zone.find_all("p"):
            txt = clean_text(p.get_text())
            if txt:
                block["text"].append(txt)

        # Listes UL
        for ul in zone.find_all("ul"):
            items = [clean_text(li.get_text()) for li in ul.find_all("li")]
            block["list"].append([i for i in items if i])

        # Ne pas ajouter les blocs vides
        if block["h3"] or block["h4"] or block["text"] or block["list"]:
            result["blocks"].append(block)

    # Extraction du texte global
    html = trafilatura.fetch_url(url)
    result["full_text"] = trafilatura.extract(html) if html else None

    return result


# ─────────────────────────────────────────────
# SCRAPER PRINCIPAL ADMISSIONS
# ─────────────────────────────────────────────
def scrape_admissions(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    data = {
        "rubric": "Admissions",
        "intro": None,
        "sections": []
    }

    # INTRO
    intro_h4 = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro_h4:
        data["intro"] = clean_text(intro_h4.get_text())

    # SUBRUBRICS
    subrubrics = []
    for card in soup.select(".post_wrapper.one_half"):

        title = card.select_one(".post_header.half h3 a")
        content = card.select_one(".post_excerpt.half p")
        btn = card.select_one(".post_excerpt.half a[href]")

        item = {
            "title": clean_text(title.get_text()) if title else None,
            "content": clean_text(content.get_text()) if content else None,
            "url": urljoin(url, btn["href"]) if btn else None,
            "child_page": None   # <-- important : on la remplira après
        }

        subrubrics.append(item)

    # CONTACTS
    contacts = []
    for c in soup.select("#page_contacts .one_fifth"):

        name = c.find("h3")
        role = c.find("h4")
        phone_raw = c.find("p")
        email_tag = c.find("a", href=lambda h: h and "mailto" in h)

        phone = None
        if phone_raw:
            phone = clean_text(phone_raw.get_text()).split()[0]

        contacts.append({
            "name": clean_text(name.get_text(separator=" ")) if name else None,
            "role": clean_text(role.get_text(separator=" ")) if role else None,
            "phone": phone,
            "email": email_tag["href"].replace("mailto:", "") if email_tag else None
        })

    # SECTION PAR H2
    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        title = clean_text(h2.get_text())
        section = {"h2": title, "content": []}

        if "procédure" in title.lower() or "procédure" in title.lower():
            section["content"] = subrubrics

        elif "contact" in title.lower():
            section["content"] = contacts

        data["sections"].append(section)

    # ─────────────────────────────────────────────
    # SCRAPER DES SOUS-PAGES
    # ─────────────────────────────────────────────
    for item in subrubrics:
        if item["url"]:
            print("→ Scraping sous-page :", item["url"])
            item["child_page"] = scrape_child_page(item["url"])

    # FULL TEXT GLOBAL
    html = trafilatura.fetch_url(url)
    data["full_text"] = trafilatura.extract(html) if html else None

    return data


# ─────────────────────────────────────────────
# EXECUTION
# ─────────────────────────────────────────────
URL = "https://www.esilv.fr/admissions/"
result = scrape_admissions(URL)

with open("admissions_full.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✔ admissions_full.json créé avec succès !")


→ Scraping sous-page : https://www.esilv.fr/admissions/concours-avenir-plus/
→ Scraping sous-page : https://www.esilv.fr/admissions/concours-avenir-prepas/
→ Scraping sous-page : https://www.esilv.fr/admissions/concours-avenir/
→ Scraping sous-page : https://www.esilv.fr/admissions/candidats-alternance/
→ Scraping sous-page : https://www.esilv.fr/admissions/concours-avenir-bachelors/
→ Scraping sous-page : https://www.esilv.fr/admissions/candidats-etrangers/
→ Scraping sous-page : https://www.esilv.fr/admissions/logement/
→ Scraping sous-page : https://www.esilv.fr/admissions/rencontrez-nous/
→ Scraping sous-page : https://www.esilv.fr/admissions/tarifs-et-financement/
✔ admissions_full.json créé avec succès !


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import trafilatura
import json
import re


def clean_text(t):
    if not t:
        return None
    t = re.sub(r"\s+", " ", t)
    return t.strip()


# ─────────────────────────────────────────────
# PARSEUR DES BLOCS "one_third" AVEC TITRE + LISTE UL
# ─────────────────────────────────────────────
def parse_titled_list(div):
    """
    Ex :
    <div class='one_third'>
        <strong>ESILV Paris (FISE) :</strong>
        <ul><li>MP : 63 places</li>...</ul>
    </div>
    """
    title_tag = div.find("strong")
    title = clean_text(title_tag.get_text()) if title_tag else None

    ul = div.find("ul")
    if not ul:
        return None

    items = [clean_text(li.get_text()) for li in ul.find_all("li")]

    return {
        "title": title,
        "items": items
    }


# ─────────────────────────────────────────────
# SCRAPER STANDARD POUR PAGE ENFANT
# ─────────────────────────────────────────────
def scrape_child_page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    result = {
        "url": url,
        "title": None,
        "intro": None,
        "blocks": []
    }

    # h2 principal
    h2 = soup.select_one(".standard_wrapper.cat_wrapper h2")
    if h2:
        result["title"] = clean_text(h2.get_text())

    # h4 introductif
    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        result["intro"] = clean_text(intro.get_text())

    # PARSE : one_half + one_third
    for section in soup.select(".one_half, .one_half.last, .one_third"):

        # Cas spécial : bloc one_third (titre + ul)
        titled_list = parse_titled_list(section)
        if titled_list:
            result["blocks"].append({
                "type": "titled_list",
                "title": titled_list["title"],
                "items": titled_list["items"]
            })
            continue

        # Sinon bloc normal
        block = {
            "h3": None,
            "h4": None,
            "text": [],
            "lists": []
        }

        h3 = section.find("h3")
        h4 = section.find("h4")

        if h3:
            block["h3"] = clean_text(h3.get_text())
        if h4:
            block["h4"] = clean_text(h4.get_text())

        # Paragraphes
        for p in section.find_all("p"):
            txt = clean_text(p.get_text())
            if txt:
                block["text"].append(txt)

        # Listes
        for ul in section.find_all("ul"):
            items = [clean_text(li.get_text()) for li in ul.find_all("li")]
            block["lists"].append(items)

        if block["h3"] or block["h4"] or block["text"] or block["lists"]:
            result["blocks"].append(block)

    # Texte global
    html = trafilatura.fetch_url(url)
    result["full_text"] = trafilatura.extract(html) if html else None

    return result



# ─────────────────────────────────────────────
# SCRAPER ADMISSIONS (PAGE PARENTE)
# ─────────────────────────────────────────────
def scrape_admissions(url):

    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    data = {
        "rubric": "Admissions",
        "intro": None,
        "sections": []
    }

    # intro
    intro_h4 = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro_h4:
        data["intro"] = clean_text(intro_h4.get_text())

    # SUBRUBRICS
    subrubrics = []
    for card in soup.select(".post_wrapper.one_half"):

        title = card.select_one(".post_header.half h3 a")
        content = card.select_one(".post_excerpt.half p")
        btn = card.select_one(".post_excerpt.half a[href]")

        subrubrics.append({
            "title": clean_text(title.get_text()) if title else None,
            "content": clean_text(content.get_text()) if content else None,
            "url": urljoin(url, btn["href"]) if btn else None,
            "child_page": None
        })

    # CONTACTS
    contacts = []
    for c in soup.select("#page_contacts .one_fifth"):

        name = c.find("h3")
        role = c.find("h4")
        phone_raw = c.find("p")
        email_tag = c.find("a", href=lambda h: h and "mailto" in h)

        phone = None
        if phone_raw:
            phone = clean_text(phone_raw.get_text()).split()[0]

        contacts.append({
            "name": clean_text(name.get_text(separator=" ")) if name else None,
            "role": clean_text(role.get_text(separator=" ")) if role else None,
            "phone": phone,
            "email": email_tag["href"].replace("mailto:", "") if email_tag else None
        })

    # SECTIONS : récupérer aussi le texte entre H2 et H2 suivant
    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        title = clean_text(h2.get_text())
        section = {"h2": title, "h3": None, "content": []}

        # si un h3 juste après
        nxt = h2.find_next_sibling()
        if nxt and nxt.name == "h3":
            section["h3"] = clean_text(nxt.get_text())

        # Cas 1 : procédures → subrubrics
        if "procédure" in title.lower():
            section["content"] = subrubrics

        # Cas 2 : contacts
        elif "contact" in title.lower():
            section["content"] = contacts

        # Cas 3 : texte brut entre sections
        else:
            cur = h2.find_next_sibling()
            while cur and cur.name not in ["h2", "hr"]:
                if cur.name == "p":
                    txt = clean_text(cur.get_text())
                    if txt:
                        section["content"].append(txt)
                cur = cur.find_next_sibling()

        data["sections"].append(section)

    # SCRAPER CHILDS
    for item in subrubrics:
        if item["url"]:
            print("Scraping child:", item["url"])
            item["child_page"] = scrape_child_page(item["url"])

    # texte complet
    html = trafilatura.fetch_url(url)
    data["full_text"] = trafilatura.extract(html) if html else None

    return data


# RUN
URL = "https://www.esilv.fr/admissions/"
result = scrape_admissions(URL)

with open("admissions_withchild.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✔ FINI !")

Scraping child: https://www.esilv.fr/admissions/concours-avenir-plus/
Scraping child: https://www.esilv.fr/admissions/concours-avenir-prepas/
Scraping child: https://www.esilv.fr/admissions/concours-avenir/
Scraping child: https://www.esilv.fr/admissions/candidats-alternance/
Scraping child: https://www.esilv.fr/admissions/concours-avenir-bachelors/
Scraping child: https://www.esilv.fr/admissions/candidats-etrangers/
Scraping child: https://www.esilv.fr/admissions/logement/
Scraping child: https://www.esilv.fr/admissions/rencontrez-nous/
Scraping child: https://www.esilv.fr/admissions/tarifs-et-financement/
✔ FINI !


In [ ]:
# ============================================================
# UTILITAIRES
# ============================================================

def clean_text(t):
    if not t:
        return None
    t = re.sub(r"\s+", " ", t)
    return t.strip()


def safe_text(tag):
    try:
        return clean_text(tag.get_text())
    except:
        return None


def fetch(url):
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"⚠️ ERROR fetching {url} → {e}")
        return None


# ============================================================
# TITLED LIST ("one_third")
# ============================================================

def parse_titled_list(div):
    try:
        title_tag = div.find("strong")
        title = safe_text(title_tag) if title_tag else None

        ul = div.find("ul")
        if not ul:
            return None

        items = [safe_text(li) for li in ul.find_all("li")]
        items = [i for i in items if i]

        return {"title": title, "items": items}
    except:
        return None


# ============================================================
# PARSEUR DE SOUS-PAGE (CHILD PAGE)
# ============================================================

def scrape_child_page(url):
    print(f"    ↳ Child page: {url}")

    html = fetch(url)
    if not html:
        return None

    soup = BeautifulSoup(html, "html.parser")

    data = {
        "url": url,
        "title": None,
        "intro": None,
        "blocks": [],
        "full_text": None
    }

    h2 = soup.select_one(".standard_wrapper.cat_wrapper h2")
    if h2:
        data["title"] = safe_text(h2)

    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        data["intro"] = safe_text(intro)

    # Blocs structurés
    for section in soup.select(".one_half, .one_third, .one_half.last"):

        titled = parse_titled_list(section)
        if titled:
            data["blocks"].append({"type": "titled_list", **titled})
            continue

        block = {"h3": None, "h4": None, "text": [], "lists": []}

        h3 = section.find("h3")
        h4 = section.find("h4")
        if h3: block["h3"] = safe_text(h3)
        if h4: block["h4"] = safe_text(h4)

        for p in section.find_all("p"):
            txt = safe_text(p)
            if txt:
                block["text"].append(txt)

        for ul in section.find_all("ul"):
            li = [safe_text(x) for x in ul.find_all("li")]
            block["lists"].append([x for x in li if x])

        if block["h3"] or block["h4"] or block["text"] or block["lists"]:
            data["blocks"].append(block)

    # Texte complet
    raw = trafilatura.fetch_url(url)
    data["full_text"] = trafilatura.extract(raw) if raw else None

    return data


# ============================================================
# PARSEUR GÉNÉRIQUE (valable pour TOUTES les rubriques)
# ============================================================

def scrape_generic_rubric(name, url):
    print(f"\n→ Scraping rubric: {name} ({url})")

    html = fetch(url)
    if not html:
        return None
    soup = BeautifulSoup(html, "html.parser")

    rubric = {
        "rubric": name,
        "intro": None,
        "sections": [],
        "child_pages": [],
        "full_text": None
    }

    # Intro H4
    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        rubric["intro"] = safe_text(intro)

    # Sous-pages détectées (h3 → liens)
    all_child_links = set()

    for a in soup.select(".post_header.half a[href]"):
        link = urljoin(url, a["href"])
        all_child_links.add(link)

    for a in soup.select(".post_excerpt a[href]"):
        link = urljoin(url, a["href"])
        all_child_links.add(link)

    # Sections H2
    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        sec = {
            "h2": safe_text(h2),
            "h3": None,
            "content": [],
            "subrubrics": [],
            "contacts": []
        }

        # H3 immédiatement après
        nxt = h2.find_next_sibling()
        if nxt and nxt.name == "h3":
            sec["h3"] = safe_text(nxt)

        # Contenu textuel entre H2 et H2 suivant
        cur = h2.find_next_sibling()
        while cur and cur.name not in ["h2", "hr"]:

            # Paragraphes
            if cur.name == "p":
                txt = safe_text(cur)
                if txt:
                    sec["content"].append(txt)

            # Divs contenant du texte
            if cur.name == "div":
                txt = safe_text(cur)
                if txt and len(txt) > 3:
                    sec["content"].append(txt)

            cur = cur.find_next_sibling()

        rubric["sections"].append(sec)

    # Child pages (même logique que Admissions)
    for link in all_child_links:
        cp = scrape_child_page(link)
        rubric["child_pages"].append(cp)

    # Texte complet
    raw = trafilatura.fetch_url(url)
    rubric["full_text"] = trafilatura.extract(raw) if raw else None

    return rubric


# ============================================================
# MANAGER PRINCIPAL
# ============================================================

START_URLS = {
    "lecole": "https://www.esilv.fr/lecole/",
    "admissions": "https://www.esilv.fr/admissions/",
    "formations": "https://www.esilv.fr/formations/",
    "entreprises-debouches": "https://www.esilv.fr/entreprises-debouches/",
    "recherche": "https://www.esilv.fr/recherche/",
    "international": "https://www.esilv.fr/international/",
}


OUTPUT_DIR = "scraping_esilv2"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def scrape_all():
    for name, url in START_URLS.items():
        try:
            print("\n============================")
            print(f" SCRAPING {name}")
            print("============================")

            data = scrape_generic_rubric(name, url)

            with open(f"{OUTPUT_DIR}/{name}.json", "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

            print(f"✔ Export → {OUTPUT_DIR}/{name}.json")

        except Exception as e:
            print(f"❌ Error scraping {name}: {e}")
            traceback.print_exc()


# LANCER
scrape_all()


 SCRAPING admissions

→ Scraping rubric: admissions (https://www.esilv.fr/admissions/)
    ↳ Child page: https://www.esilv.fr/admissions/candidats-etrangers/
    ↳ Child page: https://www.esilv.fr/admissions/rencontrez-nous/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-bachelors/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-prepas/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-plus/
    ↳ Child page: https://www.esilv.fr/admissions/tarifs-et-financement/
    ↳ Child page: https://www.esilv.fr/admissions/logement/
    ↳ Child page: https://www.esilv.fr/admissions/candidats-alternance/
✔ Export → scraping_esilv2/admissions.json


## Version officiel

In [ ]:
# ============================================================
# UTILITAIRES
# ============================================================

def clean_text(t):
    if not t:
        return None
    t = re.sub(r"\s+", " ", t)
    return t.strip()


def safe_text(tag):
    try:
        return clean_text(tag.get_text())
    except:
        return None


def fetch(url):
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        return r.text
    except Exception as e:
        print(f"⚠️ ERROR fetching {url} → {e}")
        return None


# ============================================================
# TITLED LIST ("one_third")
# ============================================================

def parse_titled_list(div):
    try:
        title_tag = div.find("strong")
        title = safe_text(title_tag) if title_tag else None

        ul = div.find("ul")
        if not ul:
            return None

        items = [safe_text(li) for li in ul.find_all("li")]
        items = [i for i in items if i]

        return {"title": title, "items": items}
    except:
        return None


# ============================================================
# PARSEUR DE SOUS-PAGE (CHILD PAGE)
# ============================================================

def scrape_child_page(url):
    print(f"    ↳ Child page: {url}")

    html = fetch(url)
    if not html:
        return None

    soup = BeautifulSoup(html, "html.parser")

    data = {
        "url": url,
        "title": None,
        "intro": None,
        "blocks": [],
        "full_text": None
    }

    h2 = soup.select_one(".standard_wrapper.cat_wrapper h2")
    if h2:
        data["title"] = safe_text(h2)

    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        data["intro"] = safe_text(intro)

    # Blocs structurés
    for section in soup.select(".one_half, .one_third, .one_half.last"):

        titled = parse_titled_list(section)
        if titled:
            data["blocks"].append({"type": "titled_list", **titled})
            continue

        block = {"h3": None, "h4": None, "text": [], "lists": []}

        h3 = section.find("h3")
        h4 = section.find("h4")
        if h3: block["h3"] = safe_text(h3)
        if h4: block["h4"] = safe_text(h4)

        for p in section.find_all("p"):
            txt = safe_text(p)
            if txt:
                block["text"].append(txt)

        for ul in section.find_all("ul"):
            li = [safe_text(x) for x in ul.find_all("li")]
            block["lists"].append([x for x in li if x])

        if block["h3"] or block["h4"] or block["text"] or block["lists"]:
            data["blocks"].append(block)

    # Texte complet
    raw = trafilatura.fetch_url(url)
    data["full_text"] = trafilatura.extract(raw) if raw else None

    return data


# ============================================================
# PARSEUR GÉNÉRIQUE (valable pour TOUTES les rubriques)
# ============================================================

def scrape_generic_rubric(name, url):
    print(f"\n→ Scraping rubric: {name} ({url})")

    html = fetch(url)
    if not html:
        return None
    soup = BeautifulSoup(html, "html.parser")

    rubric = {
        "rubric": name,
        "intro": None,
        "sections": [],
        "child_pages": [],
        "full_text": None
    }

    # Intro H4
    intro = soup.select_one(".standard_wrapper.cat_wrapper > h4")
    if intro:
        rubric["intro"] = safe_text(intro)

    # Sous-pages détectées (h3 → liens)
    all_child_links = set()

    for a in soup.select(".post_header.half a[href]"):
        link = urljoin(url, a["href"])
        all_child_links.add(link)

    for a in soup.select(".post_excerpt a[href]"):
        link = urljoin(url, a["href"])
        all_child_links.add(link)

    # Sections H2
    for h2 in soup.select(".standard_wrapper.cat_wrapper h2"):

        sec = {
            "h2": safe_text(h2),
            "h3": None,
            "content": [],
            "subrubrics": [],
            "contacts": []
        }

        # H3 immédiatement après
        nxt = h2.find_next_sibling()
        if nxt and nxt.name == "h3":
            sec["h3"] = safe_text(nxt)

        # Contenu textuel entre H2 et H2 suivant
        cur = h2.find_next_sibling()
        while cur and cur.name not in ["h2", "hr"]:

            # Paragraphes
            if cur.name == "p":
                txt = safe_text(cur)
                if txt:
                    sec["content"].append(txt)

            # Divs contenant du texte
            if cur.name == "div":
                txt = safe_text(cur)
                if txt and len(txt) > 3:
                    sec["content"].append(txt)

            cur = cur.find_next_sibling()

        rubric["sections"].append(sec)

    # Child pages (même logique que Admissions)
    for link in all_child_links:
        cp = scrape_child_page(link)
        rubric["child_pages"].append(cp)

    # Texte complet
    raw = trafilatura.fetch_url(url)
    rubric["full_text"] = trafilatura.extract(raw) if raw else None

    return rubric


# ============================================================
# MANAGER PRINCIPAL
# ============================================================

START_URLS = {
    "lecole": "https://www.esilv.fr/lecole/",
    "admissions": "https://www.esilv.fr/admissions/",
    "formations": "https://www.esilv.fr/formations/",
    "entreprises-debouches": "https://www.esilv.fr/entreprises-debouches/",
    "recherche": "https://www.esilv.fr/recherche/",
    "international": "https://www.esilv.fr/international/",
}

OUTPUT_DIR = "scraping_esilv3"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def scrape_all():
    for name, url in START_URLS.items():
        try:
            print("\n============================")
            print(f" SCRAPING {name}")
            print("============================")

            data = scrape_generic_rubric(name, url)

            with open(f"{OUTPUT_DIR}/{name}.json", "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

            print(f"✔ Export → {OUTPUT_DIR}/{name}.json")

        except Exception as e:
            print(f"❌ Error scraping {name}: {e}")
            traceback.print_exc()


# LANCER
scrape_all()


 SCRAPING admissions

→ Scraping rubric: admissions (https://www.esilv.fr/admissions/)
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-bachelors/
    ↳ Child page: https://www.esilv.fr/admissions/candidats-alternance/
    ↳ Child page: https://www.esilv.fr/admissions/rencontrez-nous/
    ↳ Child page: https://www.esilv.fr/admissions/tarifs-et-financement/
    ↳ Child page: https://www.esilv.fr/admissions/candidats-etrangers/
    ↳ Child page: https://www.esilv.fr/admissions/logement/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-prepas/
    ↳ Child page: https://www.esilv.fr/admissions/concours-avenir-plus/
✔ Export → scraping_esilv3/admissions.json
